In [20]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,Dropout,RNN,LSTMCell,BatchNormalization
from keras.optimizers import SGD

In [36]:
num_classes = 10

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train =x_train.reshape(x_train.shape[0],784)/255
x_test =x_test.reshape(x_test.shape[0],784)/255

y_train = keras.utils.to_categorical(y_train,num_classes=num_classes)
y_test = keras.utils.to_categorical(y_test,num_classes=num_classes)

In [37]:
# Перцетрпон - 3 скрытых слоя, ф-ия активации 'relu'. Функция потерь - 'adam' - адаптивный градиентный спуск (скорость обучения подгоняется автоматически во время обучения). 
# На вход вектор 1:784 далее -> 
# 1:784х784:1568            - скрытый слой 1
# 1:1568x1568:1568          - скрытый слой 2
# 1:1568x1568:1568          - скрытый слой 3
# 1:1568x1568:10 --> 1:10   - выходной слой
# 784 + 1568х3 + 10 = 5498 нейронов (узлов) всего
# 
# Быстрее работает рекурентная сеть (RNN) нежели перцептрон, точность примерно одинаковая, притом, что нейронов в RNN меньше. В данном случае точность перцептрона можно улучшить, задав функцию ошибки не adam, а sgd со скоростью обучения 0.0001 (точность ~ 98.5%). Так же можно инициализировать веса не случайными значениями, а предобученными, что тоде скажется на точности. Увеличивая число слоёв или число нейронов, можно улучшить точность, но так же повышается шанс переобучить модель, хотя в данном случае для практических задач не годится ни одна из моделей, сверточная сеть будет лучше, за счет того, что кодирует изображение "свёртками" (окошками по 9 пикселей как в лабораторной работе 6 часть 2), за счет этого, точность распознавания рукописных символов повышается вне зависимости от расположения символа на картинке.
model_1 = Sequential()
model_1.add(Dense(1568,input_dim = 784, activation = 'relu'))
model_1.add(Dropout(0.2))
model_1.add(Dense(1568, activation = 'relu'))
model_1.add(Dropout(0.2))
model_1.add(Dense(1568, activation = 'relu'))
model_1.add(Dropout(0.2))
model_1.add(Dense(num_classes, activation = 'softmax'))

model_1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_1.fit(x_train, y_train,epochs=5,batch_size=32,validation_data=(x_test, y_test))
model_1.save('lab9_SNN_1.h5')

Epoch 1/5
1875/1875 [==============================] - 65s 35ms/step - loss: 0.2343 - accuracy: 0.9310 - val_loss: 0.1252 - val_accuracy: 0.9624
Epoch 2/5
1875/1875 [==============================] - 65s 35ms/step - loss: 0.1313 - accuracy: 0.9624 - val_loss: 0.1148 - val_accuracy: 0.9683
Epoch 3/5
1875/1875 [==============================] - 64s 34ms/step - loss: 0.1033 - accuracy: 0.9706 - val_loss: 0.1022 - val_accuracy: 0.9721
Epoch 4/5
1875/1875 [==============================] - 64s 34ms/step - loss: 0.0883 - accuracy: 0.9749 - val_loss: 0.1106 - val_accuracy: 0.9717
Epoch 5/5
1875/1875 [==============================] - 67s 36ms/step - loss: 0.0809 - accuracy: 0.9774 - val_loss: 0.1262 - val_accuracy: 0.9711


In [12]:
score = model_1.evaluate(x_test, y_test, verbose=1)
print('Потери на тесте:', score[0])
print('Точность на тесте:', score[1])

313/313 [==============================] - 2s 7ms/step - loss: 0.2052 - accuracy: 0.9609
Потери на тесте: 0.2052420824766159
Точность на тесте: 0.9609000086784363


In [43]:
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
sample, sample_label = x_train[0], y_train[0]

In [44]:
lstm_layer = RNN(LSTMCell(156), input_shape = (None, 28))
model = Sequential()
model.add(lstm_layer)
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(10,activation = 'softmax'))

model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=32,epochs=5)
model.save('lab9_RNN.h5')
print("Модель сохранена как lab9_RNN.h5")

Epoch 1/5
1875/1875 [==============================] - 41s 22ms/step - loss: 1.5793 - accuracy: 0.8873 - val_loss: 1.5083 - val_accuracy: 0.9540
Epoch 2/5
1875/1875 [==============================] - 51s 27ms/step - loss: 1.5041 - accuracy: 0.9586 - val_loss: 1.4915 - val_accuracy: 0.9702
Epoch 3/5
1875/1875 [==============================] - 46s 25ms/step - loss: 1.4953 - accuracy: 0.9664 - val_loss: 1.4915 - val_accuracy: 0.9693
Epoch 4/5
1875/1875 [==============================] - 52s 28ms/step - loss: 1.4903 - accuracy: 0.9714 - val_loss: 1.4853 - val_accuracy: 0.9759
Epoch 5/5
1875/1875 [==============================] - 46s 25ms/step - loss: 1.4863 - accuracy: 0.9750 - val_loss: 1.4839 - val_accuracy: 0.9779


Модель сохранена как lab9_RNN.h5
